In [178]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
import pandas as pd
import yaml
import geopandas as gp
import time
import pandas as pd
from geopy.distance import great_circle
import folium
import h5py
from scipy.signal import butter, lfilter

from scipy.spatial import ConvexHull
from scipy.interpolate import griddata
from shapely.geometry import Point, Polygon
import csv
import math

In [179]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [180]:
flag_project = 1

if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/'

In [181]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v4.0/output_repar_v9.5_02--10-16Hz/


In [182]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [183]:
dir_CC = dir_CC_workspace + info_basic['rdir_CC']
dir_ds = dir_project + info_basic['rdir_ds']
dir_disp = dir_project + info_basic['rdir_disp_model']
dir_partition = dir_project + info_basic['rdir_partition']
dir_disp_pick = dir_project + info_basic['rdir_disp_pick']
dir_inv = dir_project + info_basic['rdir_inv_BFGS']

In [184]:
dir_surf = dir_project + info_basic['rdir_image'] + 'paper_tomo/surf/'
dir_surf_input = dir_surf + 'input/'
dir_surf_grid = dir_surf + 'grid/'
dir_surf_output = dir_surf + 'output/'
dir_origin = dir_project + info_basic['rdir_image'] + 'paper_tomo/origin/'
dir_origin_input = dir_origin + 'input/'   
dir_origin_output = dir_origin + 'output/'
if not os.path.exists(dir_surf):
    os.makedirs(dir_surf)
if not os.path.exists(dir_surf_input):
    os.makedirs(dir_surf_input)
if not os.path.exists(dir_surf_grid):
    os.makedirs(dir_surf_grid)
if not os.path.exists(dir_surf_output):
    os.makedirs(dir_surf_output)
if not os.path.exists(dir_origin):
    os.makedirs(dir_origin) 
if not os.path.exists(dir_origin_input):
    os.makedirs(dir_origin_input)
if not os.path.exists(dir_origin_output):
    os.makedirs(dir_origin_output)

In [185]:
key_subworks = info_basic['key_subworks']

In [186]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 
elevations_all =  stainfo['elevation'].tolist()

In [187]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
for key in info_basic['key_subworks']:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

In [188]:
# all stations
file = dir_project + 'stations_all_trans.txt'
stations, x_stations_all, y_stations_all = np.loadtxt(file, dtype='str' , unpack=True)
x_stations_all = x_stations_all.astype(float)
y_stations_all = y_stations_all.astype(float)

In [189]:
inter_file = dir_inv + 'vs_inter.npz'
inter = np.load(inter_file, allow_pickle=True)
vs_fund = inter['vs_fund']
vs = inter['vs']
x_inter_in = inter['x']
y_inter_in = inter['y']
lon_inter_in = inter['lon']
lat_inter_in = inter['lat']
z = inter['depth']
dz = inter['dz']
N = inter['N']

### Generate csv for stalist

In [190]:
headers = ['stations','longitude','latitude','elevation','x','y']
filename = dir_surf_input + 'stations.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(stalist_all)):
        f_csv.writerow([stalist_all[i],lon_stations_all[i],lat_stations_all[i],elevations_all[i],x_stations_all[i],y_stations_all[i]])

In [191]:
headers = ['partition','longitude','latitude']
filename = dir_surf_input + 'partition.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(lon_centroid_partition)):
        f_csv.writerow([key_subworks[i],lon_centroid_partition[i],lat_centroid_partition[i]])

In [192]:
for i in range(N):
    depth = i*dz
    headers = ['x','y','longitude','latitude','depth','vs','vs_fund']
    filename = dir_surf_input + 'vs_'+str(depth)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for j in range(len(lon_inter_in)):
            f_csv.writerow([x_inter_in[j],y_inter_in[j],lon_inter_in[j],lat_inter_in[j],depth,vs[i,j],vs_fund[i,j]])

### Phase

In [193]:
f_list = [4,8,15]
phase = {}
for f in f_list:
    phase[f] = []
for key in info_basic['key_subworks']:
    disp = np.loadtxt(dir_disp + 'disp_model_'+key+'.txt')
    disp = disp[disp[:,2] == 0]
    for f in f_list:
        phase[f].append(disp[disp[:,0]==f][0][1])

In [194]:
headers = ['longitude','latitude','4Hz','8Hz','15Hz']
filename = dir_surf_input + 'phase.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(lon_centroid_partition)):
        f_csv.writerow([lon_centroid_partition[i],lat_centroid_partition[i],phase[4][i],phase[8][i],phase[15][i]])

### Faults (lon-lat)

In [195]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

name_ref, lat_fault_infer, lon_fault_infer = np.loadtxt('clark_fault_inferred.txt',dtype='str' , unpack=True,delimiter=',')
lon_fault_infer = lon_fault_infer.astype(float)
lat_fault_infer = lat_fault_infer.astype(float)

lat_fault_wade = {}
lon_fault_wade = {}
for i in range(1,5):
    name_ref, lat_fault_wade[i], lon_fault_wade[i] = np.loadtxt('clark_fault_wade'+str(i)+'.txt',dtype='str' , unpack=True,delimiter=',')
    lon_fault_wade[i] = lon_fault_wade[i].astype(float)
    lat_fault_wade[i] = lat_fault_wade[i].astype(float)

In [196]:
for i in range(len(faults)):
    headers = ['longitude','latitude']
    filename = dir_surf_input + 'fault-latlon_'+str(i+1)+'.csv'
    # 插值
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for k in range(len(faults['clark'+str(i+1)]['lon'])):
            f_csv.writerow([faults['clark'+str(i+1)]['lon'][k],faults['clark'+str(i+1)]['lat'][k]])

In [197]:
headers = ['longitude','latitude']
filename = dir_surf_input + 'fault-latlon_infer.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(lon_fault_infer)):
        f_csv.writerow([lon_fault_infer[i],lat_fault_infer[i]])

In [198]:
for i in range(1,5):
    headers = ['longitude','latitude']
    filename = dir_surf_input + 'fault-wade-latlon_'+str(i+1)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for k in range(len(lon_fault_wade[i])):
            f_csv.writerow([lon_fault_wade[i][k],lat_fault_wade[i][k]])

### Faults (x-y)

In [199]:
# fault surface trace
faults = np.load(dir_project +'clark_faults_trans.npy', allow_pickle='TRUE').item()
fault_infer = np.load(dir_project +'clark_fault_inferred_trans.npy', allow_pickle='TRUE').item()
faults_wade = np.load(dir_project +'clark_faults_wade_trans.npy', allow_pickle='TRUE').item()

In [200]:
x_faults = {}
y_faults = {}
for i in range(len(faults)):
    x_faults['clark'+str(i+1)] = faults['clark'+str(i+1)]['x']
    y_faults['clark'+str(i+1)] = faults['clark'+str(i+1)]['y']

In [201]:
for i in range(len(faults)):
    headers = ['longitude','latitude']
    filename = dir_surf_input + 'fault_'+str(i)+'.csv'
    # 插值
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for k in range(len(faults['clark'+str(i+1)]['x'])):
            f_csv.writerow([faults['clark'+str(i+1)]['x'][k],faults['clark'+str(i+1)]['y'][k]])

In [202]:
headers = ['longitude','latitude']
filename = dir_surf_input + 'fault_infer.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(fault_infer['x'])):
        f_csv.writerow([fault_infer['x'][i],fault_infer['y'][i]])

### Faults 3D

In [203]:
for i in range(len(faults)):
    headers = ['x','y','z']
    filename = dir_origin_input + 'fault3d_'+str(i)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for k in range(len(faults['clark'+str(i+1)]['x'])):
            if faults['clark'+str(i+1)]['y'][k] >= -15 and faults['clark'+str(i+1)]['y'][k] <= 620: 
                f_csv.writerow([faults['clark'+str(i+1)]['x'][k],faults['clark'+str(i+1)]['y'][k],0])

In [204]:
headers = ['x','y','z']
filename = dir_origin_input + 'fault3d_infer.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(fault_infer['x'])):
        f_csv.writerow([fault_infer['x'][i],fault_infer['y'][i],0])

### Vs Verticle

In [205]:
inter_file = dir_inv + 'vs_inter_verti.npz'
inter = np.load(inter_file, allow_pickle=True)

In [206]:
x_inter_verti = inter['x'].tolist()
y_inter_verti = inter['y'].tolist()
vs_verti = inter['vs'].tolist()
vs_fund_verti = inter['vs_fund'].tolist()
vs_verti_pert = inter['vs_pert'].tolist()
vs_fund_verti_pert = inter['vs_fund_pert'].tolist()

In [207]:
z = np.arange(0,31) * dz
for trace in vs_verti.keys():
    x = x_inter_verti[trace]
    headers = ['x','y','z','vs','vs_fund']
    filename = dir_origin_input + 'shallow-150_vs_verti_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for i in range(len(x)):
            for k in range(len(z)):
                f_csv.writerow([x[i],y_inter_verti[trace][0],z[k],vs_verti[trace][k][i],vs_fund_verti[trace][k][i]])
for trace in vs_verti.keys():
    x = x_inter_verti[trace]
    headers = ['x','y','z','vs','vs_fund']
    filename = dir_origin_input + 'shallow-150_vs_vertipert_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for i in range(len(x)):
            for k in range(len(z)):
                f_csv.writerow([x[i],y_inter_verti[trace][0],z[k],vs_verti_pert[trace][k][i],vs_fund_verti_pert[trace][k][i]])

In [208]:
z = np.arange(0,21) * dz
for trace in vs_verti.keys():
    x = x_inter_verti[trace]
    headers = ['x','y','z','vs','vs_fund']
    filename = dir_origin_input + 'shallow-100_vs_verti_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for i in range(len(x)):
            for k in range(len(z)):
                f_csv.writerow([x[i],y_inter_verti[trace][0],z[k],vs_verti[trace][k][i],vs_fund_verti[trace][k][i]])
for trace in vs_verti.keys():
    x = x_inter_verti[trace]
    headers = ['x','y','z','vs','vs_fund']
    filename = dir_origin_input + 'shallow-100_vs_vertipert_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for i in range(len(x)):
            for k in range(len(z)):
                f_csv.writerow([x[i],y_inter_verti[trace][0],z[k],vs_verti_pert[trace][k][i],vs_fund_verti_pert[trace][k][i]])

In [209]:
z = np.arange(0,41) * dz
for trace in vs_verti.keys():
    x = x_inter_verti[trace]
    headers = ['x','y','z','vs','vs_fund']
    filename = dir_origin_input + 'shallow-200_vs_verti_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for i in range(len(x)):
            for k in range(len(z)):
                f_csv.writerow([x[i],y_inter_verti[trace][0],z[k],vs_verti[trace][k][i],vs_fund_verti[trace][k][i]])
for trace in vs_verti.keys():
    x = x_inter_verti[trace]
    headers = ['x','y','z','vs','vs_fund']
    filename = dir_origin_input + 'shallow-200_vs_vertipert_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for i in range(len(x)):
            for k in range(len(z)):
                f_csv.writerow([x[i],y_inter_verti[trace][0],z[k],vs_verti_pert[trace][k][i],vs_fund_verti_pert[trace][k][i]])

In [210]:
z = np.arange(0,37) * dz
for trace in vs_verti.keys():
    x = x_inter_verti[trace]
    headers = ['x','y','z','vs','vs_fund']
    filename = dir_origin_input + 'shallow-180_vs_verti_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for i in range(len(x)):
            for k in range(len(z)):
                f_csv.writerow([x[i],y_inter_verti[trace][0],z[k],vs_verti[trace][k][i],vs_fund_verti[trace][k][i]])
for trace in vs_verti.keys():
    x = x_inter_verti[trace]
    headers = ['x','y','z','vs','vs_fund']
    filename = dir_origin_input + 'shallow-180_vs_vertipert_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for i in range(len(x)):
            for k in range(len(z)):
                f_csv.writerow([x[i],y_inter_verti[trace][0],z[k],vs_verti_pert[trace][k][i],vs_fund_verti_pert[trace][k][i]])

### Faults inter

In [211]:
# 判断两条线段是否相交
def cross_product(p1, p2, p3):
    """计算叉积"""
    return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0])

def on_segment(p1, p2, p3):
    """检查p2是否在以p1和p3为端点的线段上"""
    print('on segement')
    return min(p1[0], p3[0]) <= p2[0] <= max(p1[0], p3[0]) and min(p1[1], p3[1]) <= p2[1] <= max(p1[1], p3[1])

def segments_intersect(p1, q1, p2, q2):
    """检查由(p1, q1)和(p2, q2)形成的两条线段是否相交"""
    # 检查线段的方向
    o1 = cross_product(p1, q1, p2)
    o2 = cross_product(p1, q1, q2)
    o3 = cross_product(p2, q2, p1)
    o4 = cross_product(p2, q2, q1)

    # 如果两条线段的方向相异，则它们相交
    if o1 * o2 < 0 and o3 * o4 < 0:
        return True
    # 特殊情况处理
    if o1 == 0 and on_segment(p1, p2, q1):
        return True
    if o2 == 0 and on_segment(p1, q2, q1):
        return True
    if o3 == 0 and on_segment(p2, p1, q2):
        return True
    if o4 == 0 and on_segment(p2, q1, q2):
        return True

    return False

def plot_segments(p1, q1, p2, q2):
    plt.plot([p1[0], q1[0]], [p1[1], q1[1]], color='blue', label='Segment 1')
    plt.plot([p2[0], q2[0]], [p2[1], q2[1]], color='red', label='Segment 2')

    intersection = segments_intersect(p1, q1, p2, q2)

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Segments Intersection')
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

In [212]:
def find_fault_inter(tag_trace,faults):
    global traces
    global x_inter_verti
    global y_inter_verti

    x_faults = {}
    y_faults = {}
    for key in faults.keys():
        x_faults[key] = faults[key]['x']
        y_faults[key] = faults[key]['y']

    x_fault_inter = []
    y_fault_inter = []
    for key in faults.keys():
        x_fault = x_faults[key]
        y_fault = y_faults[key]
        index_fault_start = np.argmin(y_fault)
        index_fault_end = np.argmax(y_fault)
        p1 = [x_fault[index_fault_start], y_fault[index_fault_start]]
        q1 = [x_fault[index_fault_end], y_fault[index_fault_end]]
        index_trace_start = np.argmin(x_inter_verti[tag_trace])
        index_trace_end = np.argmax(x_inter_verti[tag_trace])
        p2 = [x_inter_verti[tag_trace][index_trace_start], y_inter_verti[tag_trace][index_trace_start]]
        q2 = [x_inter_verti[tag_trace][index_trace_end], y_inter_verti[tag_trace][index_trace_end]]
        
        if segments_intersect(p1, q1, p2, q2):
            # find the nearest fault point
            x_trace = x_inter_verti[tag_trace]
            y_trace = y_inter_verti[tag_trace]

            min_distance = float('inf')

            for lon1, lat1 in zip(x_fault, y_fault):
                for lon2, lat2 in zip(x_trace, y_trace):
                    distance = math.sqrt((lon1 - lon2)**2 + (lat1 - lat2)**2)
                    if distance < min_distance:
                        min_distance = distance
                        x_fault_near = lon1
                        y_fault_near = lat1
                        x_trace_near = lon2
                        y_trace_near = lat2
                    
            x_fault_inter.append( x_trace_near )
            y_fault_inter.append( y_trace_near )
        
        """
        intersection = find_lines_inter(p1, q1, p2, q2)
        if intersection is not None:
            lon_fault_inter.append(intersection[0])
            lat_fault_inter.append(intersection[1])
        """
    return x_fault_inter, y_fault_inter

In [213]:
for trace in vs_verti.keys():
    x_fault, y_fault = find_fault_inter(trace,faults)
    headers = ['x','z']
    filename = dir_origin_input + 'fault_inter_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(zip(x_fault, -1*np.ones_like(x_fault)))

In [214]:
for trace in vs_verti.keys():
    x_fault_wade, y_fault_wade = find_fault_inter(trace,faults_wade)
    headers = ['x','z']
    filename = dir_origin_input + 'fault-wade_inter_trace-'+str(trace)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(zip(x_fault_wade, -1*np.ones_like(x_fault_wade)))

### Density

In [215]:
dense_partition_file = dir_project + 'density_partitions.npz'
dense_target_file = dir_project + 'density_targets.npz'
dense_partition = np.load(dense_partition_file, allow_pickle=True)
dense_target = np.load(dense_target_file, allow_pickle=True)

In [216]:
x = dense_partition['x']
y = dense_partition['y']
density_partition = dense_partition['density']
density_target = dense_target['density']

In [217]:
headers = ['x','y','density']
filename_partition = dir_surf_input + 'density_partition.csv'
filename_target = dir_surf_input + 'density_target.csv'
with open(filename_partition,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(x)):
        f_csv.writerow([x[i],y[i],density_partition[i]])
with open(filename_target,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(x)):
        f_csv.writerow([x[i],y[i],density_target[i]])